## Project 1
**Eva Fong (xsg5by),**

In [7]:
import pandas as pd
mdf = pd.read_csv('./data/linked_mortality_file_1999_2000.csv') # Load mortality file
print( mdf.head() )
gdf = pd.read_sas("./data/DEMO.xpt", format="xport") # Load demographics file
print( gdf.head() )
df = gdf.merge(mdf, on="SEQN", how="inner") # Merge mortality and demographics on SEQN variable

   SEQN  ELIGSTAT  MORTSTAT  UCOD_LEADING  DIABETES  HYPERTEN  PERMTH_INT  \
0     1         2       NaN           NaN       NaN       NaN         NaN   
1     2         1       1.0           6.0       0.0       0.0       177.0   
2     3         2       NaN           NaN       NaN       NaN         NaN   
3     4         2       NaN           NaN       NaN       NaN         NaN   
4     5         1       0.0           NaN       NaN       NaN       244.0   

   PERMTH_EXM  
0         NaN  
1       177.0  
2         NaN  
3         NaN  
4       244.0  
   SEQN  SDDSRVYR  RIDSTATR  RIDEXMON  RIAGENDR  RIDAGEYR  RIDAGEMN  RIDAGEEX  \
0   1.0       1.0       2.0       2.0       2.0       2.0      29.0      31.0   
1   2.0       1.0       2.0       2.0       1.0      77.0     926.0     926.0   
2   3.0       1.0       2.0       1.0       2.0      10.0     125.0     126.0   
3   4.0       1.0       2.0       2.0       1.0       1.0      22.0      23.0   
4   5.0       1.0       2.0       2.

#### Documentation for `ELIGSTAT`, `MORTSTAT`, `PERMTH_INT` and `RIDAGEEX`
- `ELIGSTAT`: The eligiblity status of an observation for mortality follow-up. Has three possible values of 1 for "Eligible", 2 for "Under age 18, not available for public release", and 3 for "Ineligible".

- `MORTSTAT`: The final mortality status of an observation. Has three possible values of 0 for "Assumed alive", 1 for "Assumed dead", and . for "Assumed alive, under 18, ineligible for mortality follow-up, or MCOD not available".

- `PERMTH_INT`: The number of months of follow-up from NHANES interview date. Possible integer values from 43 to 45. 

- `RIDAGEEX`: Age in months at date of examination for individuals under 85 years of age at screening.

In [8]:
body = pd.read_sas("./data/BMX.xpt", format="xport")
occupation = pd.read_sas("./data/OCQ.xpt", format="xport")
blood_pressure = pd.read_sas("./data/BPX.xpt", format="xport")
heart = pd.read_sas("./data/CDQ.xpt", format="xport")


#### Rationale for data selections

From the 2000-1999 NHANES data page, we are going to be using data on age, sex, education, income (DEMO), BMI (BMX), occupation (OCQ), blood pressure (BPQ), and heart condition (CDQ).

We chose these data because we are interested to see the influence of social-economic status on someone's mortality in addition to their health metrics. Age is, of course, is included as a part of our variable set as it is proven to be highly correlated to morality by research and by the exercises we did in class. Sex is also included to account for possible biological differences that might aid or hinder someone's life expectancy. 

Education, income, and occupation serve as the indicators for socioeconomic influences. The three together typically captures someone's access to human capital and resources available to them.

Finally, BMI, blood pressure, and heart conditions will help us identify the correlation between possible physiological risk factors and morality. 

In [9]:
df = df.merge(body, on="SEQN", how="left")
df = df.merge(occupation, on="SEQN", how="left")
df = df.merge(blood_pressure, on="SEQN", how="left")
df = df.merge(heart, on="SEQN", how="left")

In [10]:
missing = df.isnull().sum()
missing[missing > 0].sort_values(ascending=False)

BMAUPREL    9965
BMAUPLEL    9965
BMIHEAD     9964
BMALLKNE    9964
DMARACE     9963
            ... 
RIDAGEMN     173
DMDHRAGE      12
DMDHRGND      12
DMDBORN       11
DMDCITZN       3
Length: 141, dtype: int64

In the merged dataset, there are a total of 147 columns that have missing values, but that is expected given the sheer size of the dataset. 

To make documentation easier, we will only consider the missing values of the specific variables that we want to use.

In [17]:
var_set = [
    "ELIGSTAT", "MORTSTAT", "PERMTH_INT", #eligiblity, mortality status, months since follow-up
    "RIDAGEEX", #age
    "RIAGENDR", #sex
    "DMDEDUC2", #highest grade or level completed
    "INDFMPIR", #ratio of family income to poverty
    "BMXBMI", #body mass index
    "BPXSY1", #blood pressure, specifically pressure in artieries when heart beats
    "OCQ150", #type of work done last week
]

df_altered = df[var_set]
df_altered = df_altered[df_altered["ELIGSTAT"] == 1] #we will only be analyzing the participants that are eligible
missing_altered = df_altered.isnull().sum()
missing_altered[missing_altered > 0].sort_values(ascending=False)

#df_altered['RIDAGEEX'].describe()

BPXSY1      1107
INDFMPIR     841
RIDAGEEX     605
DMDEDUC2     570
BMXBMI       545
OCQ150         2
dtype: int64

#### Missing values

`BPXSY1` (blood pressure): this variable has the most missing values, likely to due participants who might've been unwilling to partake in a clinical examination

`INDFMPIR` (family income to poverty ratio): this missing values may be due to the people who provide their income to the survey

`RIDGEEX` (age): this is rather unusual, maybe a human or technical error, but should remove the observations with missing age

`BMXBMI` (BMI): again, could be missing due to participants not willing to undergo clinical examination


`OCQ150` (type of work done last week): only two missing, which means those two may just be non-responses